# Code for reading data from raw excel/csv file and storing into SQL Database

1. raw recipe dataset
2. raw interaction dataset for above recipe data for user rating on food
3. indian food dataset
4. india food-101 dataset
5. allergy dataset
6. non-veg key words

Note: making column name consistent in all tables

In [ ]:
# import all common necessary packages
import pandas as pd
import numpy as np
import json
import re
import ast

# for postgres SQL database
import psycopg2

import warnings
warnings.filterwarnings('ignore')

### Configuration

In [ ]:
import logging

VERSION = "v0.1.0"
RELEASE_DATE = "16 Jan 2024"

##### DATABASE RELATED #####
DB_CONFIG_LOCAL = {
    "dbname": "food_db",
    "host": "localhost",
    "port": 5432,
    "username": "postgres",
    "password": "postgres"
}

DB_CONFIG_CLOUD = {
    "dbname": "food_db",
    "host": "",
    "port": 5432,
    "username": "postgres",
    "password": ""
}

# select which database (local or cloud)
DB_CONFIG = DB_CONFIG_LOCAL

### Database Functions

In [ ]:
class DatabaseAccess:
    def __init__(self, db_config):
        self.conn = psycopg2.connect(
                            database=db_config['dbname'],
                            user=db_config['username'],
                            password=db_config['password'],
                            host=db_config['host'],
                            port=db_config['port'])
    def getConnection(self):
        return self.conn

### Store RAW_recipes into Database

In [ ]:
df_recipes = pd.read_csv("./data/inputs/recipes_data/RAW_recipes.csv")
print("recipes shape: ", df_recipes.shape)

recipes shape:  (231637, 12)


In [ ]:
df_recipes.columns

Index(['name', 'id', 'minutes', 'contributor_id', 'submitted', 'tags',
       'nutrition', 'n_steps', 'steps', 'description', 'ingredients',
       'n_ingredients'],
      dtype='object')

In [ ]:
df_recipes.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [ ]:
# align with column name of database
cols_rename = {
    "id": "food_id",
    "name": "food_name",
    "description": "food_description",
    "ingredients": "ingredients",
    "nutrition": "nutrition"
}
df_recipes.rename(columns=cols_rename, inplace=True)

In [ ]:
# convert into proper datatype for storing into DB
df_recipes['food_id'] = df_recipes['food_id'].astype(float)
df_recipes['nutrition'] = df_recipes['nutrition'].apply(ast.literal_eval)
df_recipes['ingredients'] = df_recipes['ingredients'].apply(ast.literal_eval)

In [ ]:
# create database connection
gbl_db_conn = DatabaseAccess(DB_CONFIG).getConnection()

def raw_recipe_save_to_db(item):
    global gbl_db_conn
    try:
        cur = gbl_db_conn.cursor()
        cur.execute("INSERT INTO public.raw_recipies (food_id, food_name, food_description, ingredients, nutrition) \
                                                VALUES (%s, %s, %s, %s, %s)", \
                                                (item.food_id, item.food_name, item.food_description, item.ingredients, item.nutrition))
        gbl_db_conn.commit()
    except (Exception, psycopg2.Error) as error:
        print("Error inserting data: ", error)
    finally:
        cur.close()

    return

In [ ]:
# store all data into DB
res = df_recipes.apply(raw_recipe_save_to_db, axis=1)

In [ ]:
# read db and validate stored dataset
df_recipes_sql = pd.read_sql_query('select * from "raw_recipies"',con=gbl_db_conn)
print(df_recipes_sql.shape)
df_recipes_sql.head()

(231637, 6)


,food_id,food_name,food_description,ingredients,nutrition,created_time
0,137739,arriba baked winter squash mexican style,autumn is my favorite time of year to cook! th...,"[winter squash, mexican seasoning, mixed spice...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",2023-06-17 00:04:53.882478
1,31490,a bit different breakfast pizza,this recipe calls for the crust to be prebaked...,"[prepared pizza crust, sausage patty, eggs, mi...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",2023-06-17 00:04:53.891878
2,112140,all in the kitchen chili,this modified version of 'mom's' chili was a h...,"[ground beef, yellow onions, diced tomatoes, t...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",2023-06-17 00:04:53.892491
3,59389,alouette potatoes,"this is a super easy, great tasting, make ahea...","[spreadable cheese with garlic and herbs, new ...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",2023-06-17 00:04:53.892945
4,44061,amish tomato ketchup for canning,my dh's amish mother raised him on this recipe...,"[tomato juice, apple cider vinegar, sugar, sal...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",2023-06-17 00:04:53.893248


In [ ]:
gbl_db_conn.close()

### Store RAW_interactions into Database

In [ ]:
df_users = pd.read_csv("./data/inputs/recipes_data/RAW_interactions.csv")
print("users shape: ", df_users.shape)

users shape:  (1132367, 5)


In [ ]:
df_users.columns

Index(['user_id', 'recipe_id', 'date', 'rating', 'review'], dtype='object')

In [ ]:
# align with column name of database
cols_rename = {
    "recipe_id": "food_id"
}
df_users.rename(columns=cols_rename, inplace=True)

In [ ]:
# create database connection
gbl_db_conn = DatabaseAccess(DB_CONFIG).getConnection()

def raw_interactions_save_to_db(item):
    global gbl_db_conn
    try:
        cur = gbl_db_conn.cursor()
        cur.execute("INSERT INTO public.raw_interactions (user_id, food_id, rating, review, date) \
                                                VALUES (%s, %s, %s, %s, %s)", \
                                                (item.user_id, item.food_id, item.rating, item.review, item.date))
        gbl_db_conn.commit()
    except (Exception, psycopg2.Error) as error:
        print("Error inserting data: ", error)
    finally:
        cur.close()

    return

In [ ]:
# convert into proper datatype for storing into DB
df_users['user_id'] = df_users['user_id'].astype(float)
df_users['food_id'] = df_users['food_id'].astype(float)
df_users['rating'] = df_users['rating'].astype(float)
df_users['date'] = pd.to_datetime(df_users['date']).dt.date

In [ ]:
# store all data into DB
res = df_users.apply(raw_interactions_save_to_db, axis=1)

In [ ]:
# read db and validate stored dataset
df_users_sql = pd.read_sql_query('select * from "raw_interactions"',con=gbl_db_conn)
print(df_users_sql.shape)
df_users_sql.head()

(1132367, 6)


,user_id,food_id,rating,review,date,created_time
0,38094,40893,4,Great with a salad. Cooked on top of stove for...,2003-02-17,2023-06-17 00:05:56.907367
1,1293707,40893,5,"So simple, so delicious! Great for chilly fall...",2011-12-21,2023-06-17 00:05:56.911362
2,8937,44394,4,This worked very well and is EASY. I used not...,2002-12-01,2023-06-17 00:05:56.912059
3,126440,85009,5,I made the Mexican topping and took it to bunk...,2010-02-27,2023-06-17 00:05:56.912696
4,57222,85009,5,"Made the cheddar bacon topping, adding a sprin...",2011-10-01,2023-06-17 00:05:56.913353


In [ ]:
gbl_db_conn.close()

### Store IndianFoodDataset into Database

In [ ]:
df_ind_food = pd.read_excel("./data/inputs/IndianFoodDataset-Clean.xlsx")
print("shape: ", df_ind_food.shape)

shape:  (6871, 6)


In [ ]:
df_ind_food.columns

Index(['Srno', 'recipename', 'ingredients', 'Cuisine', 'Course', 'Diet'], dtype='object')

In [ ]:
# align with column name of database
cols_rename = {
    "recipename": "food_name",
    "ingredients": "ingredients",
    "Cuisine": "cuisine",
    "Course": "course",
    "Diet": "diet"
}
df_ind_food.rename(columns=cols_rename, inplace=True)

In [ ]:
# create unique food id... raw recipe is uptp 560K.. so use base of 600K
df_ind_food['food_id'] = df_ind_food.index + 600000

In [ ]:
df_ind_food.head()

,Srno,food_name,ingredients,cuisine,course,diet,food_id
0,1,Masala Karela,"['Karela deseeded', 'Salt', 'Onion', 'table Gr...",Indian,Side Dish,Diabetic Friendly,600000
1,2,Spicy Tomato Rice,"['rice', 'tomatoes', 'teas Belle Bhat', 'salt ...",South Indian Recipes,Main Course,Vegetarian,600001
2,3,Ragi Semiya Upma Ragi Millet Vermicelli Brea...,"['Rice Vermicelli Noodles', 'Onion', 'Carrots ...",South Indian Recipes,South Indian Breakfast,High Protein Vegetarian,600002
3,4,Gongura Chicken Curry Andhra Style Gongura C...,"['Chicken', 'Onion ped', 'Tomato ped', 'Green ...",Andhra,Lunch,Non Vegeterian,600003
4,5,Andhra Style Alam Pachadi Adrak Chutney,"['table chana dal', 'table white urad dal', 'r...",Andhra,South Indian Breakfast,Vegetarian,600004


In [ ]:
# convert into proper datatype for storing into DB
df_ind_food['ingredients'] = df_ind_food['ingredients'].apply(ast.literal_eval)

In [ ]:
# create database connection
gbl_db_conn = DatabaseAccess(DB_CONFIG).getConnection()

def indian_food_save_to_db(item):
    global gbl_db_conn
    try:
        cur = gbl_db_conn.cursor()
        cur.execute("INSERT INTO public.indian_food (food_id, food_name, food_description, ingredients, cuisine, course, diet) \
                                                VALUES (%s, %s, %s, %s, %s, %s, %s)", \
                                                (item.food_id, item.food_name, '', item.ingredients, item.cuisine, item.course, item.diet))
        gbl_db_conn.commit()
    except (Exception, psycopg2.Error) as error:
        print("Error inserting data: ", error)
    finally:
        cur.close()

    return

In [ ]:
# store all data into DB
res = df_ind_food.apply(indian_food_save_to_db, axis=1)

In [ ]:
# read db and validate stored dataset
df_ind_food_sql = pd.read_sql_query('select * from "indian_food"', con=gbl_db_conn)
print(df_ind_food_sql.shape)
df_ind_food_sql.head()

(6871, 8)


,food_id,food_name,food_description,ingredients,cuisine,course,diet,created_time
0,600000,Masala Karela,,"[Karela deseeded, Salt, Onion, table Gram flou...",Indian,Side Dish,Diabetic Friendly,2023-06-17 00:09:59.710310
1,600001,Spicy Tomato Rice,,"[rice, tomatoes, teas Belle Bhat, salt per, te...",South Indian Recipes,Main Course,Vegetarian,2023-06-17 00:09:59.713008
2,600002,Ragi Semiya Upma Ragi Millet Vermicelli Brea...,,"[Rice Vermicelli Noodles, Onion, Carrots ped, ...",South Indian Recipes,South Indian Breakfast,High Protein Vegetarian,2023-06-17 00:09:59.713339
3,600003,Gongura Chicken Curry Andhra Style Gongura C...,,"[Chicken, Onion ped, Tomato ped, Green Chillie...",Andhra,Lunch,Non Vegeterian,2023-06-17 00:09:59.713584
4,600004,Andhra Style Alam Pachadi Adrak Chutney,,"[table chana dal, table white urad dal, red ch...",Andhra,South Indian Breakfast,Vegetarian,2023-06-17 00:09:59.713810


In [ ]:
gbl_db_conn.close()

### Store Indian_Food_101 into Database

In [ ]:
df_ind_food101 = pd.read_excel("./data/inputs/Indian_Food_101.xlsx")
print("shape: ", df_ind_food101.shape)

shape:  (255, 9)


In [ ]:
df_ind_food101.columns

Index(['name', 'ingredients', 'diet', 'prep_time', 'cook_time',
       'flavor_profile', 'course', 'state', 'region'],
      dtype='object')

In [ ]:
# align with column name of database
cols_rename = {
    "name": "food_name"
}
df_ind_food101.rename(columns=cols_rename, inplace=True)

In [ ]:
# create unique food id... raw recipe is uptp 560K.. so use base of 600K
df_ind_food101['food_id'] = df_ind_food101.index + 700000

In [ ]:
df_ind_food101.head()

,food_name,ingredients,diet,prep_time,cook_time,flavor_profile,course,state,region,food_id
0,Balu shahi,"Maida flour, yogurt, oil, sugar",vegetarian,45,25,sweet,dessert,West Bengal,East,700000
1,Boondi,"Gram flour, ghee, sugar",vegetarian,80,30,sweet,dessert,Rajasthan,West,700001
2,Gajar ka halwa,"Carrots, milk, sugar, ghee, cashews, raisins",vegetarian,15,60,sweet,dessert,Punjab,North,700002
3,Ghevar,"Flour, ghee, kewra, milk, clarified butter, su...",vegetarian,15,30,sweet,dessert,Rajasthan,West,700003
4,Gulab jamun,"Milk powder, plain flour, baking powder, ghee,...",vegetarian,15,40,sweet,dessert,West Bengal,East,700004


In [ ]:
def cleanIngredientValues(inp):
    strList =  [x.strip() for x in re.split(r',', inp)]
    return strList

df_ind_food101["ingredients"] = df_ind_food101["ingredients"].apply(lambda x: cleanIngredientValues(x))

In [ ]:
# create database connection
gbl_db_conn = DatabaseAccess(DB_CONFIG).getConnection()

def indian_food101_save_to_db(item):
    global gbl_db_conn
    try:
        cur = gbl_db_conn.cursor()
        cur.execute("INSERT INTO public.indian_food101 (food_id, food_name, food_description, ingredients, course, diet, state, region) \
                                                VALUES (%s, %s, %s, %s, %s, %s, %s, %s)", \
                                                (item.food_id, item.food_name, '', item.ingredients, item.course, item.diet, item.state, item.region))
        gbl_db_conn.commit()
    except (Exception, psycopg2.Error) as error:
        print("Error inserting data: ", error)
    finally:
        cur.close()

    return

In [ ]:
# store all data into DB
res = df_ind_food101.apply(indian_food101_save_to_db, axis=1)

In [ ]:
# read db and validate stored dataset
df_ind_food101_sql = pd.read_sql_query('select * from "indian_food101"',con=gbl_db_conn)
print(df_ind_food101_sql.shape)
df_ind_food101_sql.head()

(255, 10)


,id,food_id,food_name,food_description,ingredients,course,diet,state,region,created_time
0,1,700000,Balu shahi,,"[Maida flour, yogurt, oil, sugar]",dessert,vegetarian,West Bengal,East,2023-06-17 00:10:01.922589
1,2,700001,Boondi,,"[Gram flour, ghee, sugar]",dessert,vegetarian,Rajasthan,West,2023-06-17 00:10:01.926956
2,3,700002,Gajar ka halwa,,"[Carrots, milk, sugar, ghee, cashews, raisins]",dessert,vegetarian,Punjab,North,2023-06-17 00:10:01.927290
3,4,700003,Ghevar,,"[Flour, ghee, kewra, milk, clarified butter, s...",dessert,vegetarian,Rajasthan,West,2023-06-17 00:10:01.927599
4,5,700004,Gulab jamun,,"[Milk powder, plain flour, baking powder, ghee...",dessert,vegetarian,West Bengal,East,2023-06-17 00:10:01.927930


In [ ]:
gbl_db_conn.close()

### Store Allergy Data into Database

In [ ]:
df_allergy = pd.read_excel("./data/inputs/allergy_data.xlsx")
print("shape: ", df_allergy.shape)

shape:  (64, 2)


In [ ]:
df_allergy.columns

Index(['allergy', 'ingredients'], dtype='object')

In [ ]:
# store all value in lower case
df_allergy["allergy"] = df_allergy["allergy"].str.lower()
df_allergy["ingredients"] = df_allergy["ingredients"].str.lower()

In [ ]:
def cleanIngredientValues(inp):
    strList =  [x.strip() for x in re.split(r',', inp)]
    return strList

df_allergy["ingredients"] = df_allergy["ingredients"].apply(lambda x: cleanIngredientValues(x))

In [ ]:
# create database connection
gbl_db_conn = DatabaseAccess(DB_CONFIG).getConnection()

def allergy_data_save_to_db(item):
    global gbl_db_conn
    try:
        cur = gbl_db_conn.cursor()
        cur.execute("INSERT INTO public.allergy_data (allergy, ingredients) \
                                                VALUES (%s, %s)", \
                                                (item.allergy, item.ingredients))
        gbl_db_conn.commit()
    except (Exception, psycopg2.Error) as error:
        print("Error inserting data: ", error)
    finally:
        cur.close()

    return

In [ ]:
# store all data into DB
res = df_allergy.apply(allergy_data_save_to_db, axis=1)

In [ ]:
# read db and validate stored dataset
df_allergy_sql = pd.read_sql_query('select * from "allergy_data"',con=gbl_db_conn)
print(df_allergy_sql.shape)
df_allergy_sql.head()

(64, 3)


,allergy,ingredients,created_time
0,allium,"[asparagus, garlic, leek, nira, onion, shallot...",2023-06-17 00:10:20.198287
1,allium,[welsh],2023-06-17 00:10:20.200643
2,alpha-gal syndrome,"[cattle, deer, goat, horse, pig, sheep, rabbit]",2023-06-17 00:10:20.201501
3,aquagenic urticaria,[mineral water],2023-06-17 00:10:20.202126
4,beer,[hop],2023-06-17 00:10:20.202827


In [ ]:
gbl_db_conn.close()

### StoreNon-Veg Keywords into Database

In [ ]:
df_nonveg = pd.read_csv("./data/inputs/non_veg_keywords.csv")
print("shape: ", df_nonveg.shape)

shape:  (50, 2)


In [ ]:
df_nonveg.head()

,sno,keywords
0,0,chicken
1,1,ground beef
2,2,pork spareribs
3,3,chicken bouillon cubes
4,4,bacon


In [ ]:
# create database connection
gbl_db_conn = DatabaseAccess(DB_CONFIG).getConnection()

def nonveg_keywords_save_to_db(item):
    global gbl_db_conn
    try:
        cur = gbl_db_conn.cursor()
        cur.execute("INSERT INTO public.nonveg_keywords (sno, keywords) \
                                                VALUES (%s, %s)", \
                                                (item.sno, item.keywords))
        gbl_db_conn.commit()
    except (Exception, psycopg2.Error) as error:
        print("Error inserting data: ", error)
    finally:
        cur.close()

    return

def print_data(item):
    print(item.keywords)

In [ ]:
# store all data into DB
res = df_nonveg.apply(nonveg_keywords_save_to_db, axis=1)

In [ ]:
# read db and validate stored dataset
df_nonveg_sql = pd.read_sql_query('select * from "nonveg_keywords"',con=gbl_db_conn)
print(df_nonveg_sql.shape)
df_nonveg_sql.head()

(50, 3)


,sno,keywords,created_time
0,0,chicken,2023-06-17 00:10:36.698412
1,1,ground beef,2023-06-17 00:10:36.705028
2,2,pork spareribs,2023-06-17 00:10:36.705663
3,3,chicken bouillon cubes,2023-06-17 00:10:36.706169
4,4,bacon,2023-06-17 00:10:36.706594


In [ ]:
gbl_db_conn.close()

### Save Season data into Database

In [ ]:
df_season = pd.read_excel("./data/inputs/season_corpus.xlsx")
print("shape: ", df_season.shape)

shape:  (1266, 2)


In [ ]:
df_season.head()

,season,keywords
0,Spring,Asparagus
1,Spring,Peas
2,Spring,Radishes
3,Spring,Rhubarb
4,Spring,Fava beans


In [ ]:
# create database connection
gbl_db_conn = DatabaseAccess(DB_CONFIG).getConnection()

def season_corpus_save_to_db(item):
    global gbl_db_conn
    try:
        cur = gbl_db_conn.cursor()
        cur.execute("INSERT INTO public.season_corpus (season, keywords) \
                                                VALUES (%s, %s)", \
                                                (item.season, item.keywords))
        gbl_db_conn.commit()
    except (Exception, psycopg2.Error) as error:
        print("Error inserting data: ", error)
    finally:
        cur.close()

    return

def print_data(item):
    print(item.keywords)

In [ ]:
# store all data into DB
res = df_season.apply(season_corpus_save_to_db, axis=1)

In [ ]:
# read db and validate stored dataset
df_season_sql = pd.read_sql_query('select * from "season_corpus"',con=gbl_db_conn)
print(df_season_sql.shape)
df_season_sql.head()

(1266, 3)


,season,keywords,created_time
0,Spring,Asparagus,2023-06-17 00:11:27.429066
1,Spring,Peas,2023-06-17 00:11:27.431010
2,Spring,Radishes,2023-06-17 00:11:27.431540
3,Spring,Rhubarb,2023-06-17 00:11:27.431990
4,Spring,Fava beans,2023-06-17 00:11:27.432554


In [ ]:
gbl_db_conn.close()